In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
train_paht='../input/chest-ctscan-images/Data/train'
test_path='../input/chest-ctscan-images/Data/test'
validation_path='../input/chest-ctscan-images/Data/valid'

In [3]:
train_datagen=ImageDataGenerator(rescale=1/255, vertical_flip=True, fill_mode='reflect')
test_datagen=ImageDataGenerator(rescale=1/255,  vertical_flip=True, fill_mode='reflect')
validation_datagen=ImageDataGenerator(rescale=1/255, vertical_flip=True, fill_mode='reflect')

In [4]:
train_geneator=train_datagen.flow_from_directory(train_paht,
                                                target_size=(200,200),
                                                color_mode='grayscale',
                                                 batch_size=128,
                                                subset='training',
                                                class_mode='categorical')
test_generator=test_datagen.flow_from_directory(test_path,
                                               target_size=(200,200),
                                               color_mode='grayscale',
                                                batch_size=128,
                                               class_mode='categorical')
                                             
validation_generator=validation_datagen.flow_from_directory(validation_path,
                                                           target_size=(200,200),
                                                            color_mode='grayscale',
                                                            batch_size=128,
                                                            class_mode='categorical')

In [5]:
model=tf.keras.Sequential()
model.add(Conv2D(128, kernel_size=(3,3), padding='same', kernel_regularizer=regularizers.l1(1e-4), input_shape=(200,200,1), activation='relu'))
model.add(MaxPooling2D((2,2)))
          
          
model.add(Conv2D(128, kernel_size=(3,3), padding='same', kernel_regularizer=regularizers.l1(1e-4), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3,3), padding='same', kernel_regularizer=regularizers.l1(1e-4), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
          
model.add(Flatten())
model.add(Dense(64, kernel_regularizer=regularizers.l1(1e-5), activation='relu'))
model.add(Dropout(0.25))
          
model.add(Dense(4, activation='softmax'))
          
model.summary()     

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
history=model.fit(train_geneator, epochs=20, validation_data=validation_generator)

In [8]:
epoch=[x for x in range(1,21)]
fig, ax =plt.subplots(1,2,figsize=(12,5))
ax[0].plot(epoch, history.history["accuracy"])
ax[0].plot(epoch, history.history["val_accuracy"])
ax[0].legend(labels=["accuracy","val_accuracy"])
ax[1].plot(epoch, history.history["loss"])
ax[1].plot(epoch, history.history["val_loss"])
ax[1].legend(labels=["loss","val_loss"])
ax[0].set_xlabel("Epoch")
ax[1].set_xlabel("Epoch")
ax[0].set_ylabel("accuracy")
ax[1].set_ylabel("Loss")
fig.suptitle("Model metrics")
plt.show()

Al observar los resultados de mi primer modelo podemos inferir, que estamos observando el fenómeno de Overfitting en mi red neuronal. A razón, de la diferencia numérica observable entre mis datos de entrenamiento y validación, sumado a esto vemos unos porcentajes de perdida relativamente altos para obtener un buen modelo. En el siguiente jupyter mejoraré este modelo agregando un poco más de data augmentation, batch normalization  y una red neural preentrenada. Cabe aclarar que para mejorar el modelo y evitar este sobre ajuste podemos agregar o quitar capas a mi red.